In [2]:
from claim_decomposer import claim_decomposer
from research_agent import research_agent
from reasoning_agent import reasoning_agent
from verdict_agent import verdict_agent

In [3]:
text = """
The Apollo 11 mission landed humans on the Moon for the first time on July 20, 1969. 
Neil Armstrong was the first person to walk on the lunar surface.
The mission was launched by NASA using a Saturn V rocket.
"""

response = claim_decomposer.invoke({"text": text})
claims = response["claims"]
research_results = [research_agent.invoke({"claim": claim}) for claim in claims]

In [4]:
for claim in claims:
    print(claim)

The Apollo 11 mission landed humans on the Moon for the first time on July 20, 1969.
Neil Armstrong was the first person to walk on the lunar surface.
The Apollo 11 mission was launched by NASA using a Saturn V rocket.


In [5]:
for message in response["messages"]:
    message.pretty_print()

================================ System Message ================================

The user will give you some text. Your task is to identify and extract every factual claim made in the text.

### Instructions:

Definition: A factual claim is any statement in the document that asserts a verifiable fact about the world (e.g., data points, events, or concrete information). Do not include opinions, subjective interpretations, or ambiguous statements.

Extraction: Read the entire document carefully and list each individual factual claim. Each claim should be a complete statement that can be fact-checked.

Output Format:

Return your results as a JSON array of strings.
Each string in the array must contain one factual claim.
Do not include any additional text, explanations, or commentary—only the JSON array.

### EXAMPLE:

### Example 1
Document:
"Albert Einstein was born in Germany. He developed the theory of relativity, which revolutionized physics."

Output
[
  "Albert Einstein was born i

In [6]:
from reasoning_agent import reasoning_agent

reasoning_results = []
for result in research_results:
    claim = result["claim"]
    evidence = [evidence["result"] for evidence in result["evidence"]]
    reasoning_results.append(
        reasoning_agent.invoke({"claim": claim, "evidence": evidence})
    )

In [7]:
for result in reasoning_results:
    print(f"Claim: {result['claim']}")
    print(f"Label: {result['label']}")
    print(f"Justification: {result['justification']}")
    print(f"Evidence: {result['evidence']}")
    print("---")

Claim: The Apollo 11 mission landed humans on the Moon for the first time on July 20, 1969.
Label: true
Justification: Apollo 11's lunar module, "Eagle", successfully landed astronauts Neil Armstrong and Buzz Aldrin on the Moon on July 20, 1969, while Michael Collins orbited overhead in the Command Module, "Columbia". This marked the first time humans set foot on another celestial body.
Evidence: ["No Wikipedia page found for 'Apollo 11 mission'."]
---
Claim: Neil Armstrong was the first person to walk on the lunar surface.
Label: true
Justification: Neil Armstrong became the first person to step onto the moon's surface during the Apollo 11 mission on July 20, 1969, famously saying 'That's one small step for man, one giant leap for mankind.'
Evidence: ["No Wikipedia page found for 'Neil Armstrong first person to walk on lunar surface'."]
---
Claim: The Apollo 11 mission was launched by NASA using a Saturn V rocket.
Label: true
Justification: The Apollo 11 mission, which successfully la

In [8]:
initial_verdict_state = {
    "claims": [result["claim"] for result in reasoning_results],
    "labels": [result["label"] for result in reasoning_results],
    "justifications": [result["justification"] for result in reasoning_results],
    "messages": [],
    "final_label": None,
    "final_justification": None,
}
final_state = verdict_agent.invoke(initial_verdict_state)

In [9]:
for message in final_state["messages"]:
    message.pretty_print()

================================ System Message ================================

You are an **Automated Verdict Agent**. Your role is to analyze the individual factual evaluations of claims within a document and generate a final classification for the entire document.

### **Evaluation Guidelines:**
- **TRUE**: The majority of claims in the document are well-supported by evidence.
- **FALSE**: The majority of claims are explicitly contradicted by evidence.
- **MIXED**: The document contains a combination of true and false claims.
- **UNKNOWN**: The claims lack sufficient evidence to determine their accuracy.

### **Analysis Process:**
1. Review the claims, their verdicts, and their justifications.
2. Determine whether the majority of claims are **true, false, mixed, or unknown**.
3. Provide a **final verdict** for the entire document.
4. Generate a **clear justification** summarizing why this verdict was reached.

### **Claims Analysis:**
Claim: The Apollo 11 mission landed humans on 

In [10]:
print(final_state["final_label"])
print(final_state["final_justification"])

false
The variable 'myVar' inside the function shadows the outer one, so it doesn't refer to the global scope.
